In [6]:
import pandas as pd
from pydantic import BaseModel, ValidationError, validator
from typing import Optional, List, get_type_hints
import logging
import numpy as np

# Setting up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Base schema class with common validation logic
class BaseDataSchema(BaseModel):
    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data)

    @classmethod
    def validate_dataframe(cls, df: pd.DataFrame):
        records = df.to_dict(orient='records')
        validated_data = []
        for record in records:
            try:
                validated_record = cls.from_dict(record)
                cls.validate_mandatory_fields(validated_record)
                validated_data.append(validated_record)
            except ValidationError as e:
                logger.error(f"Validation error: {e}")
                raise
        return validated_data

    @classmethod
    def _get_validators(cls):
        type_hints = get_type_hints(cls)
        validators = {}
        for field_name, field_type in type_hints.items():
            if field_type == int:
                validators[field_name] = cls._validate_int_field
            elif field_type == str:
                validators[field_name] = cls._validate_str_field
            elif field_type == float:
                validators[field_name] = cls._validate_float_field
        return validators

    @classmethod
    def _validate_int_field(cls, v):
        try:
            return int(v)
        except (ValueError, TypeError):
            raise ValueError('Must be an integer')

    @classmethod
    def _validate_str_field(cls, v):
        if not isinstance(v, str) or v.strip() == '':
            raise ValueError('Must be a non-empty string')
        return v

    @classmethod
    def _validate_float_field(cls, v):
        try:
            return float(v)
        except (ValueError, TypeError):
            raise ValueError('Must be a float')

    @validator('*', pre=True, always=True)
    def dynamic_validator(cls, v, values, field):
        field_name = field.name
        validators = cls._get_validators()
        if field_name in validators:
            return validators[field_name](v)
        return v

    @classmethod
    def validate_mandatory_fields(cls, record):
        for field, value in record.dict().items():
            if isinstance(value, str) and value.strip() == '':
                raise ValueError(f"Mandatory field '{field}' cannot be an empty string")
            elif pd.isna(value):
                raise ValueError(f"Mandatory field '{field}' cannot be NaN")

# Base importer class with common import logic
class BaseExcelImporter(BaseDataSchema):
    _file_path: str = None

    @classmethod
    def main(cls, file_path: str):
        cls._file_path = file_path
        return cls.process_file()

    @classmethod
    def process_file(cls):
        try:
            row_data_list = cls.import_excel(cls._file_path, cls)
            logger.info("Data imported successfully.")
            return row_data_list  # Return the imported data
        except Exception as e:
            logger.error(f"An error occurred: {e}")

    @staticmethod
    def validate_file_extension(file_path: str):
        if not file_path.endswith(('.xlsx', '.xls')):
            logger.error("Invalid file format. Please upload an Excel file.")
            raise ValueError("Invalid file format. Please upload an Excel file.")

    @staticmethod
    def read_excel_file(file_path: str) -> pd.DataFrame:
        try:
            return pd.read_excel(file_path)
        except Exception as e:
            logger.error(f"Error reading the Excel file: {e}")
            raise

    @classmethod
    def validate_headers(cls, df: pd.DataFrame, expected_columns: List[str]):
        actual_columns = set(df.columns)
        if not set(expected_columns).issubset(actual_columns):
            logger.error(f"Invalid headers in the Excel file. {actual_columns}")
            raise ValueError(f"Invalid headers in the Excel file. Expected: {expected_columns}, Found: {actual_columns}")
        return df[expected_columns]

    @classmethod
    def import_excel(cls, file_path: str, schema_cls: BaseDataSchema) -> List[BaseDataSchema]:
        cls.validate_file_extension(file_path)
        df = cls.read_excel_file(file_path)
        df = cls.validate_headers(df, list(schema_cls.__annotations__.keys()))
        return schema_cls.validate_dataframe(df)

# Example handler class for specific Excel file
class ExampleExcelHandler(BaseExcelImporter):
    To_Test: int
    Name: str
    Pre_Update_URL: str
    Development_URL: Optional[str] = None
    Post_Update_URL: Optional[str] = None
    Registration: Optional[str] = None
    Login: Optional[str] = None
    Password: Optional[str] = None
    Video_Page: Optional[str] = None
    New_Test: int


# Example handler class for another specific Excel file
class ExampleExcelHandler2(BaseExcelImporter):
    To_Test: int
    search: str
    filter1: Optional[str] = None
    login: Optional[str] = None
        
        
if __name__ == "__main__":
    example_file_path = "C:\\Users\\deepa\\Documents\\qauto-deepak\\DFProjects\\qauto\\data.xlsx"
    row_data_list = ExampleExcelHandler.main(example_file_path)
    print(row_data_list)
    example_file_path1 = "C:\\Users\\deepa\\Documents\\qauto-deepak\\DFProjects\\qauto\\data 1.xlsx"
    site = ExampleExcelHandler2.main(example_file_path1)
    print(site)

INFO:__main__:Data imported successfully.
INFO:__main__:Data imported successfully.


[ExampleExcelHandler(To_Test=1, Name='scribd1', Pre_Update_URL='https://www.scribd.com/document/681293592/TCS-Knowledge-Management-Competency-test-E0-Notesmyfoot', Development_URL='nan', Post_Update_URL='nan', Registration='nan', Login='nan', Password='nan', Video_Page='nan', New_Test=1), ExampleExcelHandler(To_Test=1, Name='scribd2', Pre_Update_URL='url2', Development_URL='url2_dev', Post_Update_URL='nan', Registration='nan', Login='nan', Password='nan', Video_Page='url2_video', New_Test=1)]
[ExampleExcelHandler2(To_Test=1, search='abc', filter1='def', login='ghi'), ExampleExcelHandler2(To_Test=1, search='jkl', filter1='nan', login='nan')]
